In [ ]:
import os
import requests

# Programs
programs = ["PERM", "H2-A", "H2-B", "LCA"]
# Years
years = list(range(2008, 2025))
# Start with the most recent year
# years.reverse()
# Quarters
quarters = ["Q1", "Q2", "Q3", "Q4"]
# Start with the most recent quarter
# quarters.reverse()

def download_file(url, save_dir, file_name):
    # Create the directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    # Send a GET request to the URL
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Define the file path
        file_path = os.path.join(save_dir, file_name)
        # Write the content to the file
        with open(file_path, 'wb') as f:
            f.write(response.content)
        return True
    else:
        return False

# Download directory
DOWNLOAD_DIR = "/Users/mitchv34/Work/DataDrivenLaborInsights/LCA_Disclosure_Skills/data/raw/"

# Loop through all combinations of programs, years, and quarters
for p in programs[:1]:  # Limit to the first program for testing
    for y in years[:1]:  # Limit to the first year for testing
        quarters_downloaded = []
        for q in quarters:
            # Construct the URL and file name
            url = f"https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/{p}_Disclosure_Data_FY{y}_{q}.xlsx"
            file_name = DOWNLOAD_DIR + f"{p}_Disclosure_Data_FY{y}_{q}.xlsx"
            # Attempt to download the file
            if download_file(url, "/Users/mitchv34/Work/DataDrivenLaborInsights/LCA_Disclosure_Skills/data/raw", file_name):
                quarters_downloaded.append(q)
                print(f"Downloaded {url}")
            else:
                print(f"Failed to download {url}")
            
        if len(quarters_downloaded) == 0:
            print(f"No quarters downloaded for {p} in {y}")
            # Download the file without the quarter if no quarters were downloaded
            url = f"https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/{p}_FY{y}.xlsx"
            file_name = DOWNLOAD_DIR + f"{p}_Disclosure_Data_FY{y}.xlsx"
            if download_file(url, "/Users/mitchv34/Work/DataDrivenLaborInsights/LCA_Disclosure_Skills/data/raw", file_name):
                print(f"Downloaded {url}")
            else:
                print(f"Failed to download {url}")

Failed to download https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2008_Q1.xlsx
Failed to download https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2008_Q2.xlsx
Failed to download https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2008_Q3.xlsx
Failed to download https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2008_Q4.xlsx
No quarters downloaded for PERM in 2008
Downloaded https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_FY2008.xlsx


In [ ]:
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2008.xlsx
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/H2A_FY2008.xlsx
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_FY2008.xlsx